In [22]:
from glob import glob
import os
import mne
import numpy as np
import pandas
import matplotlib.pyplot as plt

In [2]:
all_files_path = glob("dataverse_files/*.edf")
print(len(all_files_path))

6


In [3]:
all_files_path[0]

'dataverse_files/h01.edf'

In [4]:
healthy_file_path = [i for i in all_files_path if 'h' in i.split('/')[1]]
patient_file_path = [i for i in all_files_path if 's' in i.split('/')[1]]
print(len(healthy_file_path),len(patient_file_path))

3 3


In [5]:
def readData(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5, h_freq=45)
    epochs = mne.make_fixed_length_epochs(data, duration=5, overlap=1)
    array = epochs.get_data()
    return array

In [6]:
%%capture
sample_data = readData(healthy_file_path[2])

In [7]:
sample_data.shape #no of epochs, channels, length of signal

(231, 19, 1250)

In [8]:
%%capture
control_epochs_array = [readData(i) for i in healthy_file_path]

patient_epochs_array = [readData(i) for i in patient_file_path]

In [9]:
print(len(control_epochs_array))
patient_epochs_array[0].shape

3


(240, 19, 1250)

In [10]:
%%capture
control_epochs_labels = [len(i)*[0] for i in control_epochs_array]

patient_epochs_labels = [len(i)*[1] for i in patient_epochs_array]

In [11]:
len(control_epochs_labels), len(patient_epochs_labels)

(3, 3)

In [12]:
data_list = control_epochs_array + patient_epochs_array
label_list = control_epochs_labels + patient_epochs_labels

In [13]:
group_list = [[i]*len(j) for i,j in enumerate(data_list)]
# len(group_list)
len(label_list)

6

In [14]:
data_array = np.vstack(data_list)
label_array = np.hstack(label_list)
group_array = np.hstack(group_list)
print(data_array.shape, label_array.shape, group_array.shape)

(1426, 19, 1250) (1426,) (1426,)


In [15]:
from scipy import stats

def mean(x):
    return np.mean(x, axis=-1)

def std(x):
    return np.std(x, axis=-1)

def ptp(x):
    return np.ptp(x, axis=-1)

def var(x):
    return np.var(x, axis=-1)

def minim(x):
    return np.min(x, axis=-1)

def maxim(x):
    return np.max(x, axis=-1)

def argminim(x):
    return np.argmin(x, axis=-1)

def argmaxim(x):
    return np.argmax(x, axis=-1)

def rms(x):
    return np.sqrt(np.mean(x**2, axis=-1))

def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x, axis=-1)),axis=-1)

def skewness(x):
    return stats.skew(x, axis=-1)

def kurtosis(x):
    return stats.kurtosis(x, axis=-1)

def concatenate_features(x):
    return np.concatenate((mean(x), std(x),ptp(x),var(x),minim(x),
            maxim(x),argminim(x),argmaxim(x),rms(x),abs_diff_signal(x),
            skewness(x),kurtosis(x)), axis=-1)


In [16]:
features = []
for d in data_array:
    features.append(concatenate_features(d))

In [17]:
features_array = np.array(features)
features_array.shape

(1426, 228)

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV

In [19]:
clf = LogisticRegression()
gkf = GroupKFold(5)
pipe = Pipeline([('scaler',StandardScaler()), ('clf',clf)])
param_grid = {'clf__C':[0.1,0.5,0.7,1,3,5,7]}
gscv = GridSearchCV(pipe, param_grid, cv=gkf, n_jobs=12)
gscv.fit(features_array, label_array, groups=group_array)

In [20]:
gscv.best_score_

0.6578647151934823

# Using Deep Learning ( 1 D Convolution Neural Network)

In [21]:
epochs_array = np.vstack(data_list)
epochs_labels = np.hstack(label_list)
groups_array = np.hstack(group_list)

In [22]:
epochs_array.shape, epochs_labels.shape, groups_array.shape

((1426, 19, 1250), (1426,), (1426,))

In [23]:
epochs_array = np.moveaxis(epochs_array, 1, 2)
epochs_array.shape

(1426, 1250, 19)

In [1]:
%%capture
from tensorflow.keras.layers import Conv1D, BatchNormalization, LeakyReLU, MaxPool1D,\
GlobalAveragePooling1D, Dense, Dropout, AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session

2023-02-06 20:14:21.610577: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 20:14:22.278582: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-06 20:14:22.278644: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-06 20:14:24.974339: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
def Model():
    clear_session()
    model = Sequential()

    model.add(Conv1D(filters=5, kernel_size=3, strides=1, input_shape=(1250, 19)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2, strides=2))

    model.add(Conv1D(filters=4, kernel_size=3, strides=1))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2, strides=2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters=4, kernel_size=3, strides=1))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2, strides=2))
    model.add(Dropout(0.5))

    model.add(Conv1D(filters=3, kernel_size=3, strides=1))
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2, strides=2))

    model.add(Conv1D(filters=3, kernel_size=3, strides=1))
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())
    model.add(Dense(1, activation='sigmoid'))

    model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [4]:
model = Model()
model.summary()

2023-02-06 20:14:41.011679: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-06 20:14:41.011724: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-06 20:14:41.011760: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (azad-hplaptop15da0xxx): /proc/driver/nvidia/version does not exist
2023-02-06 20:14:41.012288: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1248, 5)           290       
                                                                 
 batch_normalization (BatchN  (None, 1248, 5)          20        
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 1248, 5)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 624, 5)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 622, 4)            64        
                                                                 
 batch_normalization_1 (Batc  (None, 622, 4)           1

In [5]:
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf = GroupKFold()

In [30]:
accuracy = []
for train_index,val_index in gkf.split(epochs_array, epochs_labels, groups=group_array):
    train_features, train_labels = epochs_array[train_index], epochs_labels[train_index]
    val_features, val_labels = epochs_array[val_index], epochs_labels[val_index]
    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1,\
        train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1,\
        val_features.shape[-1])).reshape(val_features.shape)    
    model = Model()
    model.load_weights("mmodel.h5")
    model.fit(train_features, train_labels, epochs=10, batch_size=20, validation_data=(val_features, val_labels))
    accuracy.append(model.evaluate(val_features, val_labels)[1])
    break

Epoch 1/10
57/57 [==============================] - 7s 48ms/step - loss: 0.0092 - accuracy: 0.9956 - val_loss: 0.8616 - val_accuracy: 0.8706
Epoch 2/10
57/57 [==============================] - 2s 40ms/step - loss: 0.0060 - accuracy: 0.9991 - val_loss: 0.8455 - val_accuracy: 0.8951
Epoch 3/10
57/57 [==============================] - 3s 45ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 1.4528 - val_accuracy: 0.7622
Epoch 4/10
57/57 [==============================] - 2s 30ms/step - loss: 0.0121 - accuracy: 0.9947 - val_loss: 1.2412 - val_accuracy: 0.7727
Epoch 5/10
57/57 [==============================] - 2s 32ms/step - loss: 0.0119 - accuracy: 0.9965 - val_loss: 1.0672 - val_accuracy: 0.8881
Epoch 6/10
57/57 [==============================] - 2s 31ms/step - loss: 0.0064 - accuracy: 0.9982 - val_loss: 1.7718 - val_accuracy: 0.6818
Epoch 7/10
57/57 [==============================] - 2s 31ms/step - loss: 0.0042 - accuracy: 0.9991 - val_loss: 1.4286 - val_accuracy: 0.8357
Epoch 8/10
57

In [31]:
np.mean(accuracy)

0.8566433787345886

In [32]:
model.save("mmodel.h5")

In [33]:
train_features.shape

(1140, 1250, 19)

In [34]:
# (1140, 1250, 19) -> (1140*1250,19)

# ChronoNet: A Deep Recurrent Nural Network for Abnormal EEG Identification


In [31]:
from IPython.display import Image
Image(url='https://www.researchgate.net/publication/322886158/figure/fig3/AS:614142884974634@1523434482761/Proposed-ChronoNet-architecture-which-includes-both-multiple-filters-of-exponentially.png')

In [43]:
import torch.nn as nn
import torch
from sklearn.model_selection import GroupKFold
input = torch.randn(3,14,512)
input.shape

torch.Size([3, 14, 512])

In [2]:
nn.Conv1d(in_channels=22, out_channels=32, kernel_size=2, stride=2, padding=0)

Conv1d(22, 32, kernel_size=(2,), stride=(2,))

In [3]:
class Block(nn.Module):
    def __init__(self, inplace):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=inplace, out_channels=32, kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv1d(in_channels=inplace, out_channels=32, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv1d(in_channels=inplace, out_channels=32, kernel_size=8, stride=2, padding=3)
        self.relu = nn.ReLU()
        
    def forward(self,x):
        x1 = self.relu(self.conv1(x))
        x2 = self.relu(self.conv2(x))
        x3 = self.relu(self.conv3(x))
        x = torch.cat([x1,x2,x3], dim=1)
        return x  

In [4]:
block = Block(14)
out1 = block(input)
out1.shape

torch.Size([3, 96, 256])

In [5]:
block = Block(96)
out2 = block(out1)
out2.shape

torch.Size([3, 96, 128])

In [6]:
block = Block(96)
out3 = block(out2)
out3.shape

torch.Size([3, 96, 64])

In [7]:
gru = nn.GRU(input_size=96, hidden_size=32, batch_first=True)
gru1 = nn.GRU(input_size=96, hidden_size=32, batch_first=True)
gru2 = nn.GRU(input_size=32, hidden_size=32, batch_first=True)

x = out3.permute(0, 2, 1)
print(x.shape)
gru_output, hn = gru1(x)
print(gru_output.shape, hn.shape)

torch.Size([3, 64, 96])
torch.Size([3, 64, 32]) torch.Size([1, 3, 32])


In [8]:
gru1_output, _ = gru1(x)
gru2_output, _ = gru2(gru1_output)

gru_out = torch.cat([gru1_output, gru2_output], dim=2)
gru_out.shape


torch.Size([3, 64, 64])

In [9]:
gru3_layer = nn.GRU(input_size=64, hidden_size=32, batch_first=True)
gru3_output, _ = gru3_layer(gru_out)
gru3_output.shape

torch.Size([3, 64, 32])

In [10]:
gru_out = torch.cat([gru1_output, gru2_output, gru3_output], dim=2)
gru_out.shape

torch.Size([3, 64, 96])

In [11]:
gru4 = nn.GRU(input_size=96, hidden_size=32, batch_first=True)
gru4_out, _ = gru4(gru_out)
gru4_out.shape, _.shape

(torch.Size([3, 64, 32]), torch.Size([1, 3, 32]))

In [12]:
linear_layer = nn.Linear(64, 1)
linear_out = linear_layer(gru_out.permute(0,2,1))
linear_out.shape

torch.Size([3, 96, 1])

In [13]:
gru4 = nn.GRU(input_size=96, hidden_size=32, batch_first=True)
gru4_out, _ = gru4(linear_out.permute(0,2,1))
gru4_out.shape

torch.Size([3, 1, 32])

In [14]:
class ChronoNet(nn.Module):
    def __init__(self,channel):
        super().__init__()
        self.block1 = Block(channel)
        self.block2 = Block(96)
        self.block3 = Block(96)
        self.gru1 = nn.GRU(input_size=96, hidden_size=32, batch_first=True)
        self.gru2 = nn.GRU(input_size=32, hidden_size=32, batch_first=True)
        self.gru3_layer = nn.GRU(input_size=64, hidden_size=32, batch_first=True)                    
        self.gru4 = nn.GRU(input_size=96, hidden_size=32, batch_first=True)                    
        self.linear_layer = nn.Linear(64, 1)
        self.flatten_layer = nn.Flatten() 
        self.fc1 = nn.Linear(32,1) 
        self.relu = nn.ReLU()
                            
    def forward(self, x):
        x = self.block1(x)  
        x = self.block2(x) 
        x = self.block3(x) 
        x = x.permute(0, 2, 1)
        gru1_output, _ = self.gru1(x)
        gru2_output, _ = self.gru2(gru1_output)
        gru_out = torch.cat([gru1_output, gru2_output], dim=2)
        gru3_output, _ = self.gru3_layer(gru_out)
        gru_out = torch.cat([gru1_output, gru2_output, gru3_output], dim=2)
#         print("gru_out",gru_out.shape)
        linear_out = self.relu(self.linear_layer(gru_out.permute(0,2,1)))
        gru4_out, _ = self.gru4(linear_out.permute(0,2,1))
        out = self.flatten_layer(gru4_out)
        out = self.fc1(out)
        return out                    
        

In [15]:
model = ChronoNet(14)

In [16]:
out = model(input)
out.shape

torch.Size([3, 1])

In [17]:
%%capture
TDC_PATH = "/home/azadm/Desktop/Brain_Computer_Interface_Model/EEG in schizophrenia/EEG dataset of individuals with intellectual and developmental disorder and healthy controls while observing rest and music stimuli/Data/CleanData/CleanData_TDC/Rest"
IDD_PATH = "/home/azadm/Desktop/Brain_Computer_Interface_Model/EEG in schizophrenia/EEG dataset of individuals with intellectual and developmental disorder and healthy controls while observing rest and music stimuli/Data/CleanData/CleanData_IDD/Rest"
!rm "/home/azadm/Desktop/Brain_Computer_Interface_Model/EEG in schizophrenia/EEG dataset of individuals with intellectual and developmental disorder and healthy controls while observing rest and music stimuli/Data/CleanData/CleanData_IDD/Rest/NDS001_Rest_CD(1).mat"

In [18]:
import scipy.io

In [23]:
# data = []
for idd in glob(IDD_PATH+"/*.mat"):
    data = scipy.io.loadmat(idd)
    break

In [24]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'clean_data'])

In [25]:
data = data['clean_data']
data.shape       #this EEG signal have total 14 channel

(14, 15360)

In [26]:
n_channels = 14
sampling_freq = 128  #in Hertz
info = mne.create_info(n_channels, sfreq=sampling_freq)
print(info)

<Info | 7 non-empty values
 bads: []
 ch_names: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13
 chs: 14 misc
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 64.0 Hz
 meas_date: unspecified
 nchan: 14
 projs: []
 sfreq: 128.0 Hz
>


In [27]:
ch_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
ch_types = ['eeg'] * 14
info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
info.set_montage('standard_1020')

<Info | 8 non-empty values
 bads: []
 ch_names: AF3, F7, F3, FC5, T7, P7, O1, O2, P8, T8, FC6, F4, F8, AF4
 chs: 14 EEG
 custom_ref_applied: False
 dig: 17 items (3 Cardinal, 14 EEG)
 highpass: 0.0 Hz
 lowpass: 64.0 Hz
 meas_date: unspecified
 nchan: 14
 projs: []
 sfreq: 128.0 Hz
>

In [28]:
%%capture
raw = mne.io.RawArray(data, info)
raw.filter(l_freq=1, h_freq=30, fir_design='firwin')

In [29]:
epochs = mne.make_fixed_length_epochs(raw, duration=4, overlap=0)

Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


In [30]:
epochs.get_data().shape

Using data from preloaded Raw for 30 events and 512 original time points ...
0 bad epochs dropped


(30, 14, 512)

In [31]:
128*4

512

In [32]:
def convertMat2mne(data):
    ch_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
    ch_types = ['eeg'] * 14
    info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
    info.set_montage('standard_1020')
    raw = mne.io.RawArray(data, info)
    raw.filter(l_freq=1, h_freq=30, fir_design='firwin')
    raw.set_eeg_reference()
    epochs = mne.make_fixed_length_epochs(raw, duration=4, overlap=0)
    return epochs.get_data()
    

In [33]:
%%capture
idd_subject = []
for idd in glob(IDD_PATH+"/*.mat"):
    data = scipy.io.loadmat(idd)['clean_data']
    data = convertMat2mne(data)
    idd_subject.append(data)

In [34]:
%%capture
tdc_subject = []
for tdc in glob(TDC_PATH+"/*.mat"):
    data = scipy.io.loadmat(tdc)['clean_data']
    data = convertMat2mne(data)
    tdc_subject.append(data)

In [35]:
len(idd_subject),len(tdc_subject)

(7, 7)

In [36]:
idd_subject[0].shape

(30, 14, 512)

In [37]:
control_epochs_labels = [len(i)*[0] for i in tdc_subject]
patients_epochs_labels = [len(i)*[1] for i in idd_subject]
print(len(control_epochs_labels), len(patients_epochs_labels))

7 7


In [38]:
data_list = tdc_subject + idd_subject
label_list = control_epochs_labels + patients_epochs_labels
print(len(data_list), len(label_list))

14 14


In [39]:
group_list = [[i]*len(j) for i,j in enumerate(data_list)]
len(group_list)

14

In [40]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import StandardScaler

class StandardScaler3D(BaseEstimator, TransformerMixin):
    #batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()
    
    def fit(self, X, y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self
    
    def transform(self, X):
        return self.scaler.transform(X.reshape(-1, X.shape[2])).reshape(X.shape)
    

In [41]:
import numpy as np
data_array = np.concatenate(data_list)
label_array = np.concatenate(label_list)
group_array = np.concatenate(group_list)
data_array = np.moveaxis(data_array, 1, 2)
print(data_array.shape, label_array.shape, group_array.shape)

(420, 512, 14) (420,) (420,)


In [44]:
gkf = GroupKFold()
accuracy = []
for train_index,val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features, train_labels = data_array[train_index], label_array[train_index]
    val_features, val_labels = data_array[val_index], label_array[val_index]
    scaler = StandardScaler3D()
    train_features = scaler.fit_transform(train_features)
    val_features = scaler.transform(val_features)
    train_features = np.moveaxis(train_features, 1, 2)
    val_features = np.moveaxis(val_features, 1, 2)
    break

In [45]:
train_features = torch.Tensor(train_features)
val_features = torch.Tensor(val_features)
train_labels = torch.Tensor(train_labels)
val_labels = torch.Tensor(val_labels)

In [46]:
print(train_features.shape)
len(val_features), len(train_features)

torch.Size([330, 14, 512])


(90, 330)

In [55]:
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable


In [56]:
from pytorch_lightning import LightningModule, Trainer
import torchmetrics
from torch.utils.data import TensorDataset, DataLoader

In [70]:
class ChronoModel(LightningModule):
    def __init__(self):
        super(ChronoModel, self).__init__()
        self.model = ChronoNet(14)
        self.lr = 1e-3
        self.bs = 12
        self.worker = 2
        self.acc = torchmetrics.Accuracy(task='binary')
        self.creterion = nn.BCEWithLogitsLoss()
        
    def forward(self, x):
        x = self.model(x)
        return x
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)
    
    def train_dataloader(self):
        dataset = TensorDataset(train_features, train_labels)
        loader = DataLoader(dataset, batch_size=self.bs, num_workers=self.worker, suffle=True)
        return loader
    
    def training_step(self, batch, batch_idx):
        signal, label = batch
        out = self(signal.float())
        loss = self.creterion(out.flatten(), label.float().flatten())
        acc = self.acc(out.flatten(), label.long().flatten())
        return {'loss': loss, 'acc': acc}
    
    def trained_epoch_end(self, outputs):
        acc = torch.stack([x['acc'] for x in outputs]).mean().detach().cpu().numpy().round(2)
        loss = torch.stack([x['loss'] for x in outputs]).mean().detach().cpu().numpy().round(2)
        print('train_acc: ',acc,' train_loss: ',loss)
        
    def val_dataloader(self):
        dataset = TensorDataset(val_features, val_labels)
        loader = DataLoader(dataset, batch_size=self.bs, num_workers=self.worker, suffle=True)
        return loader
    
    def validation_step(self, batch, batch_idx):
        signal, label = batch
        out = self(signal.float())
        loss = self.creterion(out.flatten(), label.float().flatten())
        acc = self.acc(out.flatten(), label.long().flatten())
        return {'loss': loss, 'acc': acc}
    
    def validation_epoch_end(self, outputs):
        acc = torch.stack([x['acc'] for x in outputs]).mean().detach().cpu().numpy().round(2)
        loss = torch.stack([x['loss'] for x in outputs]).mean().detach().cpu().numpy().round(2)
        print('val_acc: ',acc,' va_loss: ',loss)

In [71]:
model = ChronoModel()

In [72]:
trainer = Trainer(max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [75]:
trainer.fit(model)


  | Name      | Type              | Params
------------------------------------------------
0 | model     | ChronoNet         | 133 K 
1 | acc       | BinaryAccuracy    | 0     
2 | creterion | BCEWithLogitsLoss | 0     
------------------------------------------------
133 K     Trainable params
0         Non-trainable params
133 K     Total params
0.534     Total estimated model params size (MB)


AttributeError: module 'lib' has no attribute 'X509_V_FLAG_CB_ISSUER_CHECK'